# CCC (Continuously Changing Corruptions) Evaluation Notebook

This notebook sets up the CCC repository, installs dependencies, and runs evaluations for test-time adaptation models.

## Python Version Requirements

**Recommended: Python 3.9 or 3.10**

The code uses:
- f-strings (Python 3.6+)
- Type hints (Python 3.5+)
- PyTorch 1.9+ (supports Python 3.6-3.10)

**Best choice: Python 3.9 or 3.10** for optimal compatibility with PyTorch and all dependencies.

## Overview
- Check Python version
- Clone the CCC repository
- Install required dependencies (with compatible versions)
- Run evaluations for different models and configurations


In [5]:
# Check Python version
import sys

python_version = sys.version_info
print(f"Python version: {python_version.major}.{python_version.minor}.{python_version.micro}")
print(f"Python version info: {sys.version}")

# Check if Python version is compatible
if python_version.major == 3 and python_version.minor >= 8:
    if python_version.minor >= 9:
        print("✓ Python version is compatible (3.9+ recommended)")
    else:
        print("⚠ Python 3.8 detected. Python 3.9 or 3.10 is recommended for best compatibility.")
else:
    print("✗ Python version may not be compatible. Please use Python 3.8 or higher (3.9-3.10 recommended).")
    print("  The code requires Python 3.6+ but PyTorch works best with Python 3.8-3.10.")

print("\n" + "="*60)



Python version: 3.10.19
Python version info: 3.10.19 (main, Oct 21 2025, 16:37:10) [Clang 20.1.8 ]
✓ Python version is compatible (3.9+ recommended)



## Clone Repository

Clone the CCC repository if not already present.


In [ ]:
# Check if repository is already cloned, if not clone it
import os
import subprocess

# Determine the base directory (where the notebook is located)
base_dir = os.getcwd()
# If we're already in CCC directory, go up one level
if os.path.basename(base_dir) == "CCC":
    base_dir = os.path.dirname(base_dir)

repo_name = "CCC"
repo_path = os.path.join(base_dir, repo_name)

if not os.path.exists(repo_path):
    print("Cloning CCC repository...")
    subprocess.run(["git", "clone", "https://github.com/oripress/CCC.git"], cwd=base_dir, check=True)
    print("Repository cloned successfully!")
else:
    print(f"Repository already exists at {repo_path}")

# Change to the CCC directory for running evaluations
os.chdir(repo_path)
print(f"Current directory: {os.getcwd()}")
print(f"Repository path: {repo_path}")


Repository already exists at /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC
Current directory: /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC
Repository path: /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC


## 🔧 GPU Setup and Installation

**⚠ IMPORTANT: This notebook must run on RunPod, not your local Mac!**

**You're currently running on:** Your local Mac (no GPU support)  
**You need to run on:** RunPod server (has GPU)

### Option 1: Run Jupyter on RunPod (RECOMMENDED)

1. **SSH into RunPod:**
   ```bash
   ssh -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io
   ```

2. **On RunPod, install Jupyter:**
   ```bash
   pip install jupyter notebook
   ```

3. **Start Jupyter on RunPod:**
   ```bash
   jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root
   ```

4. **Access from your Mac:**
   - Copy the URL with token from RunPod terminal
   - Or use port forwarding: `ssh -L 8888:localhost:8888 -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io`
   - Then open `http://localhost:8888` in your browser

### Option 2: Use RunPod's Built-in Jupyter

If RunPod has Jupyter pre-installed, access it through RunPod's web interface.

**Then run the cell below on RunPod to setup GPU.**


In [ ]:
# ============================================================================
# SINGLE CELL: GPU Setup and Verification
# This cell handles everything needed to connect to GPU
# ============================================================================

import sys
import subprocess
import os
import platform

print("="*70)
print("GPU SETUP AND VERIFICATION")
print("="*70)

# Step 0: Check if running on correct system
print("\n[Step 0] System Check...")
system = platform.system()
hostname = platform.node()
python_path = sys.executable

print(f"  System: {system}")
print(f"  Hostname: {hostname}")
print(f"  Python: {python_path}")

# Check if running on Mac (local) vs Linux (RunPod)
if system == "Darwin" or "miniconda3" in python_path or "Users" in python_path:
    print("\n" + "="*70)
    print("⚠ WARNING: You're running on your LOCAL MAC!")
    print("="*70)
    print("\nMacs don't support CUDA/GPU. You need to run this on RunPod!")
    print("\nTo fix this:")
    print("  1. SSH into RunPod:")
    print("     ssh -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io")
    print("\n  2. On RunPod, start Jupyter:")
    print("     pip install jupyter")
    print("     jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser")
    print("\n  3. Access from your Mac using port forwarding:")
    print("     ssh -L 8888:localhost:8888 -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io")
    print("     Then open http://localhost:8888 in browser")
    print("\n  4. Upload this notebook to RunPod and run it there")
    print("="*70)
    raise RuntimeError("Cannot install CUDA PyTorch on Mac. Please run on RunPod.")

# Step 1: Check GPU Hardware
print("\n[Step 1] Checking GPU Hardware...")
try:
    result = subprocess.run(["nvidia-smi", "--query-gpu=name,driver_version", "--format=csv,noheader"], 
                           capture_output=True, text=True, timeout=10)
    if result.returncode == 0:
        gpu_info = result.stdout.strip().split('\n')[0]
        print(f"  ✓ GPU detected: {gpu_info}")
        gpu_available = True
    else:
        print("  ⚠ nvidia-smi not available")
        gpu_available = False
except Exception as e:
    print(f"  ⚠ Could not check GPU: {e}")
    gpu_available = False

# Step 2: Check Python Environment
print(f"\n[Step 2] Python Environment:")
print(f"  Python: {sys.executable}")
print(f"  Version: {sys.version.split()[0]}")

# Step 3: Check Current PyTorch Installation
print(f"\n[Step 3] Checking Current PyTorch Installation...")
need_install = False  # Initialize variable
torch_installed = False
cuda_available = False

try:
    import torch
    torch_installed = True
    print(f"  ✓ PyTorch version: {torch.__version__}")
    print(f"  ✓ PyTorch location: {torch.__file__}")
    
    # Check for CUDA version mismatch
    try:
        import torchvision
        print(f"  ✓ Torchvision version: {torchvision.__version__}")
        
        # Try to check CUDA versions
        try:
            torch_cuda = torch.version.cuda if torch.cuda.is_available() else None
            # This might raise an error if versions don't match
            cuda_available = torch.cuda.is_available()
            
            if cuda_available:
                print(f"  ✓ CUDA version: {torch.version.cuda}")
                print(f"  ✓ GPU: {torch.cuda.get_device_name(0)}")
                
                # Test if there's a version mismatch by trying to use torchvision
                try:
                    # This will fail if versions don't match
                    test_img = torchvision.transforms.ToTensor()(torch.zeros(3, 224, 224))
                    print("\n" + "="*70)
                    print("🎉 SUCCESS! GPU is already working!")
                    print("="*70)
                    print("\nYou can proceed with evaluation. No installation needed.")
                    need_install = False
                except RuntimeError as e:
                    if "CUDA versions" in str(e) or "different CUDA versions" in str(e):
                        print("  ⚠ CUDA version mismatch detected!")
                        print(f"  Error: {e}")
                        print("  Need to reinstall torchvision to match PyTorch CUDA version")
                        need_install = True
                    else:
                        raise
            else:
                print("  ⚠ CUDA not available - need to install CUDA PyTorch")
                need_install = True
        except RuntimeError as e:
            if "CUDA versions" in str(e) or "different CUDA versions" in str(e):
                print("  ⚠ CUDA version mismatch detected!")
                print(f"  Error: {e}")
                print("  Need to reinstall torchvision to match PyTorch CUDA version")
                need_install = True
            else:
                print(f"  ⚠ Error checking CUDA: {e}")
                need_install = True
    except ImportError:
        print("  ⚠ Torchvision not installed")
        if torch.cuda.is_available():
            print("  PyTorch has CUDA but torchvision missing - will install")
            need_install = True
        else:
            need_install = True
            
except ImportError:
    print("  ⚠ PyTorch not installed")
    need_install = True
    torch_installed = False

# Step 4: Install CUDA PyTorch if needed
if need_install:
    print(f"\n[Step 4] Installing CUDA-enabled PyTorch...")
    print("  This will install PyTorch 2.0.1 with CUDA 11.8 support")
    print("  (CUDA 11.8 is compatible with CUDA 12.x systems)")
    
    # Uninstall existing PyTorch first
    if torch_installed:
        print("  Uninstalling existing PyTorch...")
        subprocess.run([sys.executable, "-m", "pip", "uninstall", "torch", "torchvision", "-y", "-q"], 
                      check=False, capture_output=True)
    
    # Install NumPy first (required)
    print("  Installing NumPy < 2.0...")
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "numpy<2.0", "-q"], 
                      check=True, capture_output=True, timeout=60)
    except:
        pass
    
    # Check if we just need to fix torchvision version mismatch
    if torch_installed and cuda_available:
        print("  Fixing CUDA version mismatch...")
        print("  Reinstalling torchvision to match PyTorch CUDA version...")
        try:
            # Uninstall torchvision first
            subprocess.run([sys.executable, "-m", "pip", "uninstall", "torchvision", "-y", "-q"], 
                          check=False, capture_output=True)
            # Reinstall torchvision matching PyTorch's CUDA version
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", "torchvision==0.15.2",
                 "--index-url", "https://download.pytorch.org/whl/cu118"],
                check=True,
                capture_output=True,
                text=True,
                timeout=300
            )
            print("    ✓ Torchvision reinstalled successfully!")
            success = True
        except Exception as e:
            print(f"    ✗ Failed to fix version mismatch: {str(e)[:100]}")
            print("    Will try full reinstall...")
            success = False
    else:
        success = False
    
    # Install CUDA PyTorch - try multiple methods
    if not success:
        print("  Installing PyTorch with CUDA 11.8...")
        
        # Method 1: Using --index-url (recommended)
        try:
            print("    Trying method 1: --index-url...")
            result = subprocess.run(
                [sys.executable, "-m", "pip", "install", "torch==2.0.1", "torchvision==0.15.2",
                 "--index-url", "https://download.pytorch.org/whl/cu118"],
                check=True,
                capture_output=True,
                text=True,
                timeout=600
            )
            print("    ✓ Installation successful!")
            success = True
        except subprocess.TimeoutExpired:
            print("    ✗ Timeout - trying alternative method...")
        except subprocess.CalledProcessError as e:
            print(f"    ✗ Failed: {str(e)[:100]}")
            print("    Trying alternative method...")
        
        # Method 2: Using extra-index-url
        if not success:
            try:
                print("    Trying method 2: --extra-index-url...")
                result = subprocess.run(
                    [sys.executable, "-m", "pip", "install", "torch==2.0.1", "torchvision==0.15.2",
                     "--extra-index-url", "https://download.pytorch.org/whl/cu118"],
                    check=True,
                    capture_output=True,
                    text=True,
                    timeout=600
                )
                print("    ✓ Installation successful!")
                success = True
            except Exception as e:
                print(f"    ✗ Failed: {str(e)[:100]}")
        
        # Method 3: Install separately to ensure matching versions
        if not success:
            print("    Trying method 3: Separate installation...")
            try:
                # Install PyTorch first
                subprocess.run(
                    [sys.executable, "-m", "pip", "install", "torch==2.0.1",
                     "--index-url", "https://download.pytorch.org/whl/cu118"],
                    check=True,
                    capture_output=True,
                    timeout=300
                )
                # Then install matching torchvision
                subprocess.run(
                    [sys.executable, "-m", "pip", "install", "torchvision==0.15.2",
                     "--index-url", "https://download.pytorch.org/whl/cu118"],
                    check=True,
                    capture_output=True,
                    timeout=300
                )
                print("    ✓ Installation successful!")
                success = True
            except Exception as e:
                print(f"    ✗ Failed: {str(e)[:100]}")
    
    if not success:
        print("\n" + "="*70)
        print("⚠ INSTALLATION FAILED")
        print("="*70)
        print("\nPlease install manually in terminal:")
        print(f"  {sys.executable} -m pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118")
        print("\nThen restart kernel and run this cell again.")
        raise RuntimeError("CUDA PyTorch installation failed")
    
    print("\n  ✓ CUDA PyTorch installed successfully!")
    print("  ⚠ IMPORTANT: Restarting kernel to load new PyTorch...")
    print("  (You may need to manually restart: Kernel → Restart)")

# Step 5: Final Verification
print(f"\n[Step 5] Final GPU Verification...")
print("  (If you just installed, restart kernel first!)")

try:
    # Force reload if just installed
    if need_install:
        import importlib
        if 'torch' in sys.modules:
            importlib.reload(sys.modules['torch'])
    
    import torch
    cuda_available = torch.cuda.is_available()
    
    if cuda_available:
        print(f"  ✓ CUDA available: True")
        print(f"  ✓ CUDA version: {torch.version.cuda}")
        print(f"  ✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"  ✓ Number of GPUs: {torch.cuda.device_count()}")
        
        # Test GPU with actual computation
        try:
            print("  Testing GPU computation...")
            test_tensor = torch.randn(100, 100).cuda()
            result = test_tensor @ test_tensor
            print("  ✓ GPU computation test: SUCCESS")
        except Exception as e:
            print(f"  ✗ GPU computation test failed: {e}")
            raise
        
        print("\n" + "="*70)
        print("🎉 SUCCESS! GPU IS FULLY CONNECTED AND WORKING!")
        print("="*70)
        print("\n✓ GPU hardware detected")
        print("✓ CUDA PyTorch installed")
        print("✓ GPU computation verified")
        print("\nYou can now proceed with evaluation cells.")
        print("="*70)
    else:
        print("  ✗ CUDA still not available")
        print("\n" + "="*70)
        print("⚠ GPU NOT DETECTED")
        print("="*70)
        print("\nPossible issues:")
        print("  1. Kernel not restarted after installation")
        print("     → Go to: Kernel → Restart & Clear Output")
        print("     → Then run this cell again")
        print("\n  2. Python environment mismatch")
        print(f"     → Terminal Python: Check with 'which python3'")
        print(f"     → Notebook Python: {sys.executable}")
        print("     → They should match!")
        print("\n  3. CUDA libraries not found")
        print("     → Check: ls /usr/local/cuda*/lib64")
        print("="*70)
        
except Exception as e:
    print(f"  ✗ Error during verification: {e}")
    print("\nPlease restart kernel and run this cell again.")


GPU SETUP AND VERIFICATION

[Step 0] System Check...
  System: Darwin
  Hostname: dhcp-128-189-237-153.ubcsecure.wireless.ubc.ca
  Python: /Users/himanshumishra/miniconda3/envs/py310/bin/python

⚠ WARNING: You're running on your LOCAL MAC!

Macs don't support CUDA/GPU. You need to run this on RunPod!

To fix this:
  1. SSH into RunPod:
     ssh -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io

  2. On RunPod, start Jupyter:
     pip install jupyter
     jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser

  3. Access from your Mac using port forwarding:
     ssh -L 8888:localhost:8888 -i ~/.ssh/runpod_key eqdlc2mhm8ogbt-64411dd7@ssh.runpod.io
     Then open http://localhost:8888 in browser

  4. Upload this notebook to RunPod and run it there


RuntimeError: Cannot install CUDA PyTorch on Mac. Please run on RunPod.

## ⚠ IMPORTANT: GPU Detection

**If you installed CUDA PyTorch in the terminal, you MUST restart the Jupyter kernel!**

The kernel needs to be restarted to load the new CUDA-enabled PyTorch. Otherwise, it will still use the old CPU version that was loaded into memory.

**To restart:** Go to `Kernel` → `Restart` (or `Restart & Clear Output`)


In [ ]:
# Check if CUDA PyTorch is installed and GPU is detected
# Run this cell AFTER restarting the kernel if you installed CUDA PyTorch in terminal

import sys
import subprocess
import os

print("Checking GPU/CUDA status and Python environment...")
print("="*60)

# Check Python environment
print("Python Environment:")
print(f"  Python executable: {sys.executable}")
print(f"  Python version: {sys.version.split()[0]}")
print(f"  Python path: {sys.executable}")

# Check which pip is being used
try:
    pip_result = subprocess.run([sys.executable, "-m", "pip", "--version"], 
                               capture_output=True, text=True, timeout=5)
    print(f"  Pip: {pip_result.stdout.strip()}")
except:
    print("  ⚠ Could not check pip version")

# Check nvidia-smi
print("\nGPU Hardware Check:")
try:
    nvidia_result = subprocess.run(["nvidia-smi", "--query-gpu=name", "--format=csv,noheader"], 
                                   capture_output=True, text=True, timeout=5)
    if nvidia_result.returncode == 0:
        gpu_name = nvidia_result.stdout.strip().split('\n')[0]
        print(f"  ✓ GPU detected: {gpu_name}")
    else:
        print("  ⚠ nvidia-smi not available")
except:
    print("  ⚠ nvidia-smi not available")

# Check PyTorch installation
print("\nPyTorch Installation:")
try:
    import torch
    print(f"  ✓ PyTorch version: {torch.__version__}")
    print(f"  ✓ PyTorch location: {torch.__file__}")
    
    # Check if it's CPU or CUDA build
    print(f"  ✓ CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        print(f"  ✓ CUDA version: {torch.version.cuda}")
        print(f"  ✓ GPU: {torch.cuda.get_device_name(0)}")
        print(f"  ✓ Number of GPUs: {torch.cuda.device_count()}")
        
        # Test GPU with a simple operation
        try:
            test_tensor = torch.randn(10, 10).cuda()
            _ = test_tensor @ test_tensor
            print("  ✓ GPU test: SUCCESS")
        except Exception as e:
            print(f"  ✗ GPU test failed: {e}")
        
        print("\n" + "="*60)
        print("🎉 SUCCESS! GPU is detected and ready to use!")
        print("You can proceed with the evaluation cells.")
        print("="*60)
    else:
        print("\n" + "="*60)
        print("⚠ WARNING: CUDA is NOT available in this kernel!")
        print("="*60)
        print("\nDiagnostics:")
        print(f"  - PyTorch location: {torch.__file__}")
        print(f"  - Python executable: {sys.executable}")
        
        # Check if CUDA libraries are available
        print("\nChecking CUDA libraries...")
        cuda_lib_paths = [
            "/usr/local/cuda/lib64",
            "/usr/lib/x86_64-linux-gnu",
            "/usr/local/cuda-11.8/lib64",
            "/usr/local/cuda-12.1/lib64",
        ]
        cuda_found = False
        for path in cuda_lib_paths:
            if os.path.exists(path):
                print(f"  ✓ Found CUDA path: {path}")
                cuda_found = True
        if not cuda_found:
            print("  ⚠ CUDA library paths not found")
        
        print("\nPossible solutions:")
        print("  1. Make sure you installed CUDA PyTorch in the SAME Python environment")
        print(f"     Terminal Python: Check with 'which python3' in terminal")
        print(f"     Notebook Python: {sys.executable}")
        print("     They should match!")
        print("\n  2. If Python paths don't match:")
        print("     - Install CUDA PyTorch using the notebook's Python:")
        print(f"       {sys.executable} -m pip install torch==2.0.1 torchvision==0.15.2 --index-url https://download.pytorch.org/whl/cu118")
        print("\n  3. Or restart kernel and run installation cell")
        print("="*60)
        
except ImportError:
    print("  ✗ PyTorch is not installed")
    print("\n" + "="*60)
    print("Please run the installation cell first.")
    print("="*60)


Checking GPU/CUDA status and Python environment...
Python Environment:
  Python executable: /Users/himanshumishra/miniconda3/envs/py310/bin/python
  Python version: 3.10.19
  Python path: /Users/himanshumishra/miniconda3/envs/py310/bin/python
  Pip: pip 25.2 from /Users/himanshumishra/miniconda3/envs/py310/lib/python3.10/site-packages/pip (python 3.10)

GPU Hardware Check:
  ⚠ nvidia-smi not available

PyTorch Installation:
  ✓ PyTorch version: 2.0.1
  ✓ PyTorch location: /Users/himanshumishra/miniconda3/envs/py310/lib/python3.10/site-packages/torch/__init__.py
  ✓ CUDA available: False

⚠ WARNING: CUDA is NOT available in this kernel!

Diagnostics:
  - PyTorch location: /Users/himanshumishra/miniconda3/envs/py310/lib/python3.10/site-packages/torch/__init__.py
  - Python executable: /Users/himanshumishra/miniconda3/envs/py310/bin/python

Checking CUDA libraries...
  ⚠ CUDA library paths not found

Possible solutions:
  1. Make sure you installed CUDA PyTorch in the SAME Python environm

## Install Dependencies

Install all required packages with compatible versions. **Important**: We need NumPy < 2.0 for compatibility with torchvision.


In [ ]:
# Install dependencies with compatible versions
import sys
import subprocess

print("Installing dependencies with compatible versions...")
print("="*60)

# First, uninstall existing packages to avoid conflicts
print("Step 1: Cleaning up existing installations...")
try:
    subprocess.run([sys.executable, "-m", "pip", "uninstall", "torch", "torchvision", "numpy", "-y", "-q"], 
                  check=False, capture_output=True)
    print("  ✓ Cleaned up existing installations")
except:
    pass

# Install NumPy first (must be < 2.0 for compatibility)
print("\nStep 2: Installing NumPy < 2.0 (required for compatibility)...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "numpy<2.0", "-q"],
        check=True,
        capture_output=True
    )
    print("  ✓ NumPy < 2.0 installed")
except Exception as e:
    print(f"  ✗ Error installing NumPy: {e}")

# Install PyTorch and torchvision together with compatible versions
print("\nStep 3: Installing PyTorch and torchvision (compatible versions)...")
try:
    subprocess.run(
        [sys.executable, "-m", "pip", "install", "torch==2.0.1", "torchvision==0.15.2", "-q"],
        check=True,
        capture_output=True
    )
    print("  ✓ PyTorch 2.0.1 and torchvision 0.15.2 installed successfully")
except subprocess.CalledProcessError as e:
    print(f"  ⚠ Error with specific versions, trying compatible range...")
    try:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", "torch>=2.0.0,<2.1.0", "torchvision>=0.15.0,<0.16.0", "-q"],
            check=True,
            capture_output=True
        )
        print("  ✓ PyTorch and torchvision installed (compatible versions)")
    except:
        print("  ✗ Failed to install PyTorch/torchvision")
        raise

# Install other dependencies
print("\nStep 4: Installing other dependencies...")
other_packages = [
    "webdataset>=0.2.0",
    "Pillow>=8.0.0",
]

for package in other_packages:
    print(f"  Installing {package}...")
    try:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", package, "-q"],
            check=True,
            capture_output=True
        )
        print(f"  ✓ {package} installed")
    except subprocess.CalledProcessError as e:
        print(f"  ✗ Error installing {package}")

print("\n" + "="*60)
print("Dependency installation completed!")
print("="*60)


Installing dependencies with CUDA support for GPU...
⚠ GPU not detected via nvidia-smi
  Will still attempt CUDA installation - PyTorch will detect GPU if available

Step 1: Cleaning up existing installations...
  ✓ Cleaned up existing installations

Step 2: Installing NumPy < 2.0 (required for compatibility)...
  ✓ NumPy < 2.0 installed

Step 3: Installing PyTorch and torchvision with CUDA support...
  Trying CUDA 12.1 first (for CUDA 12.x systems like RunPod)...
  ⚠ Error with CUDA 12.1, trying CUDA 11.8...
  ⚠ Trying CPU version as fallback...
  ✓ PyTorch 2.0.1 and torchvision 0.15.2 installed (CPU version)
  ⚠ WARNING: CPU version installed. GPU will not be available!

Step 4: Installing other dependencies...
  Installing webdataset>=0.2.0...
  ✓ webdataset>=0.2.0 installed
  Installing Pillow>=8.0.0...
  ✓ Pillow>=8.0.0 installed

Dependency installation completed!


## Verify Installations

Check that all packages are installed correctly and are compatible.


## Create Logs Directory

Create directory for storing evaluation results.


In [ ]:
# Verify installations and compatibility
print("Verifying installations...")
print("="*60)

try:
    import torch
    print(f"✓ PyTorch version: {torch.__version__}")
except ImportError as e:
    print(f"✗ PyTorch not installed: {e}")
    torch = None

try:
    import torchvision
    print(f"✓ Torchvision version: {torchvision.__version__}")
    
    # Test compatibility by importing models
    try:
        import torchvision.models as models
        print("✓ Torchvision models import successful")
    except Exception as e:
        print(f"✗ Torchvision compatibility issue: {e}")
        
except ImportError as e:
    print(f"✗ Torchvision not installed: {e}")

try:
    import numpy
    print(f"✓ NumPy version: {numpy.__version__}")
    if numpy.__version__.startswith("2."):
        print("  ⚠ WARNING: NumPy 2.x detected. This may cause compatibility issues.")
        print("  Please downgrade to NumPy < 2.0 by running: pip install 'numpy<2.0'")
except ImportError as e:
    print(f"✗ NumPy not installed: {e}")

try:
    import webdataset
    print(f"✓ WebDataset version: {webdataset.__version__}")
except ImportError as e:
    print(f"✗ WebDataset not installed: {e}")

try:
    import PIL
    print(f"✓ Pillow version: {PIL.__version__}")
except ImportError as e:
    print(f"✗ Pillow not installed: {e}")

if torch is not None:
    print(f"\nCUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"CUDA version: {torch.version.cuda}")
        print(f"GPU device: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠ CUDA not available. GPU acceleration will not be used.")
        print("  Note: The evaluation can run on CPU but will be VERY slow.")

print("="*60)


Logs directory already exists: logs
Created dataset directory: ccc_datasets

Dataset will be stored at: /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC/ccc_datasets
Logs will be stored at: /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC/logs


In [ ]:
# Create logs directory
import os

logs_dir = "logs"
if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)
    print(f"Created logs directory: {logs_dir}")
else:
    print(f"Logs directory already exists: {logs_dir}")


Baseline: 20 (CCC-Medium)
Storage location: /Users/himanshumishra/Library/CloudStorage/OneDrive-UBC/UBC/Term1/ProjectAdaptive/CCC/ccc_datasets
Note: This will download the dataset files. The dataset is large and may take time.
You can skip this step and use streaming if you prefer (slower but no download needed).

[1/9] Downloading: baseline_20_transition+speed_1000_seed_43
------------------------------------------------------------


KeyboardInterrupt: 

## Run Evaluations

Run evaluations for different models. The evaluation will:
- Stream datasets from cloud (no local download needed)
- Evaluate on different baseline accuracies (0, 20, 40)
- Test different models (rdumb, tent, pretrained, etc.)
- **Run all computations on GPU** (requires CUDA)

**Note**: Each evaluation processes 9 runs (3 seeds × 3 transition speeds). 

**GPU Performance**: With GPU, each evaluation should complete in a reasonable time.


## Run Evaluations

Run evaluations for different models. The evaluation will:
- Use locally stored datasets (downloaded in previous step)
- Evaluate on different baseline accuracies (0, 20, 40)
- Test different models (rdumb, tent, pretrained, etc.)

**Note**: Each evaluation processes 9 runs (3 seeds × 3 transition speeds). 

**GPU Performance**: With GPU, each evaluation should complete in a reasonable time.


## Run Evaluations

Run evaluations for different models. The evaluation will:
- Use streaming datasets (no local download needed)
- Evaluate on different baseline accuracies (0, 20, 40)
- Test different models (rdumb, tent, pretrained, etc.)

**Note**: Each evaluation processes 9 runs (3 seeds × 3 transition speeds). This may take some time.

**Important**: 
- The evaluation can run on CPU but will be VERY slow (may take days)
- For reasonable performance, use a system with GPU and CUDA installed


In [ ]:
# Run evaluation for RDumb model (the paper's main contribution)
# This will evaluate on baseline 20 (CCC-Medium) with all 9 configurations

import subprocess
import sys
import os

# Ensure we're in the CCC directory
if not os.path.exists("eval.py"):
    print("Error: eval.py not found. Make sure you're in the CCC directory.")
    print(f"Current directory: {os.getcwd()}")
else:
    # Configuration
    mode = "rdumb"
    baseline = 20  # CCC-Medium (0=Hard, 20=Medium, 40=Easy)
    logs_path = "logs"

    print(f"Running evaluation for mode: {mode}, baseline: {baseline}")
    print("This will evaluate all 9 configurations (3 seeds × 3 speeds)")
    print("Note: This may take a while as it processes the entire dataset...")
    print("⚠ WARNING: If running on CPU, this may take days to complete!")
    print("\n" + "="*60)

    # Run evaluation for each of the 9 configurations
    # processind 0-8 covers all combinations
    for processind in range(9):
        print(f"\nRunning evaluation {processind + 1}/9 (processind={processind})")
        print("-"*60)
        
        cmd = [
            sys.executable, "eval.py",
            "--mode", mode,
            "--baseline", str(baseline),
            "--logs", logs_path,
            "--processind", str(processind),
            "--dset", ""  # Empty since we're using streaming
        ]
        
        try:
            result = subprocess.run(cmd, check=True, capture_output=True, text=True, timeout=3600)
            print(f"✓ Completed processind {processind}")
            if result.stdout:
                # Only print last few lines to avoid clutter
                lines = result.stdout.strip().split('\n')
                if len(lines) > 5:
                    print("  ... (output truncated) ...")
                    for line in lines[-3:]:
                        print(f"  {line}")
                else:
                    print(result.stdout)
        except subprocess.TimeoutExpired:
            print(f"✗ Timeout for processind {processind} (took longer than 1 hour)")
            print("  This is normal for CPU execution. Consider using GPU for faster results.")
            break
        except subprocess.CalledProcessError as e:
            print(f"✗ Error in processind {processind}")
            print(f"Error output (last 500 chars):")
            error_msg = e.stderr if e.stderr else e.stdout
            if error_msg:
                print(error_msg[-500:])
            else:
                print("No error message available")
            # Don't break, continue with next processind
            continue

    print("\n" + "="*60)
    print("Evaluation completed!")
    print("="*60)


Running evaluation for mode: rdumb, baseline: 20
This will evaluate all 9 configurations (3 seeds × 3 speeds)
Note: This may take a while as it processes the entire dataset...
⚠ WARNING: If running on CPU, this may take days to complete!


Running evaluation 1/9 (processind=0)
------------------------------------------------------------
✗ Error in processind 0
Error output (last 500 chars):
 in read
    raise ReadError("unexpected end of data")
tarfile.ReadError: ("unexpected end of data @ <Pipe ((['curl', '--connect-timeout', '30', '--retry', '30', '--retry-delay', '2', '-f', '-s', '-L', 'https://mlcloud.uni-tuebingen.de:7443/datasets/CCC/baseline_20_transition+speed_1000_seed_43/serial_00000.tar'],), {'bufsize': 8192})>", <webdataset.gopen.Pipe object at 0x13d140400>, 'https://mlcloud.uni-tuebingen.de:7443/datasets/CCC/baseline_20_transition+speed_1000_seed_43/serial_00000.tar')


Running evaluation 2/9 (processind=1)
------------------------------------------------------------


python(55131) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


KeyboardInterrupt: 

In [ ]:
# List all result files
import os
import glob

results_dir = os.path.join("logs", "ccc_20")
if os.path.exists(results_dir):
    result_files = glob.glob(os.path.join(results_dir, "*.txt"))
    print(f"Found {len(result_files)} result files:")
    for f in sorted(result_files):
        print(f"  - {os.path.basename(f)}")
        
    # Read and display a sample result file
    if result_files:
        print("\n" + "="*60)
        print("Sample result file (first 20 lines):")
        print("="*60)
        with open(result_files[0], 'r') as f:
            lines = f.readlines()[:20]
            for i, line in enumerate(lines, 1):
                print(f"{i:4d}: {line.strip()}")
else:
    print(f"Results directory not found: {results_dir}")


In [ ]:
# Calculate average accuracy from result files
import numpy as np
import glob
import os

def calculate_avg_accuracy(result_file):
    """Calculate average accuracy from a result file."""
    accuracies = []
    with open(result_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('acc_'):
                try:
                    acc = float(line.split('_')[1])
                    accuracies.append(acc)
                except:
                    pass
    return np.mean(accuracies) if accuracies else None

# Calculate averages for all result files
results_dir = os.path.join("logs", "ccc_20")
if os.path.exists(results_dir):
    result_files = glob.glob(os.path.join(results_dir, "*.txt"))
    
    print("Average Accuracies:")
    print("="*60)
    for f in sorted(result_files):
        avg_acc = calculate_avg_accuracy(f)
        if avg_acc is not None:
            filename = os.path.basename(f)
            print(f"{filename:60s} {avg_acc:.4f}%")
    
    # Overall average
    all_accs = [calculate_avg_accuracy(f) for f in result_files]
    all_accs = [a for a in all_accs if a is not None]
    if all_accs:
        print("="*60)
        print(f"{'Overall Average:':60s} {np.mean(all_accs):.4f}%")
        print(f"{'Std Dev:':60s} {np.std(all_accs):.4f}%")
else:
    print(f"Results directory not found: {results_dir}")


In [ ]:
# List all result files
import os
import glob

results_dir = os.path.join("logs", "ccc_20")
if os.path.exists(results_dir):
    result_files = glob.glob(os.path.join(results_dir, "*.txt"))
    print(f"Found {len(result_files)} result files:")
    for f in sorted(result_files):
        print(f"  - {os.path.basename(f)}")
        
    # Read and display a sample result file
    if result_files:
        print("\n" + "="*60)
        print("Sample result file (first 20 lines):")
        print("="*60)
        with open(result_files[0], 'r') as f:
            lines = f.readlines()[:20]
            for i, line in enumerate(lines, 1):
                print(f"{i:4d}: {line.strip()}")
else:
    print(f"Results directory not found: {results_dir}")


Found 8 result files:
  - model_rdumb_baseline_20_transition+speed_1000_seed_43.txt
  - model_rdumb_baseline_20_transition+speed_1000_seed_44.txt
  - model_rdumb_baseline_20_transition+speed_1000_seed_45.txt
  - model_rdumb_baseline_20_transition+speed_2000_seed_43.txt
  - model_rdumb_baseline_20_transition+speed_2000_seed_44.txt
  - model_rdumb_baseline_20_transition+speed_2000_seed_45.txt
  - model_rdumb_baseline_20_transition+speed_5000_seed_43.txt
  - model_rdumb_baseline_20_transition+speed_5000_seed_44.txt

Sample result file (first 20 lines):
   1: acc_28.1250000000
   2: acc_29.6875000000
   3: acc_32.8125000000
   4: acc_25.0000000000
   5: acc_29.6875000000
   6: acc_29.6875000000
   7: acc_34.3750000000
   8: acc_23.4375000000
   9: acc_34.3750000000
  10: acc_25.0000000000
  11: acc_23.4375000000
  12: acc_34.3750000000
  13: acc_31.2500000000
  14: acc_26.5625000000
  15: acc_40.6250000000
  16: acc_34.3750000000
  17: acc_29.6875000000
  18: acc_29.6875000000
  19: acc_48

In [ ]:
# Calculate average accuracy from result files
import numpy as np
import glob
import os

def calculate_avg_accuracy(result_file):
    """Calculate average accuracy from a result file."""
    accuracies = []
    with open(result_file, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('acc_'):
                try:
                    acc = float(line.split('_')[1])
                    accuracies.append(acc)
                except:
                    pass
    return np.mean(accuracies) if accuracies else None

# Calculate averages for all result files
results_dir = os.path.join("logs", "ccc_20")
if os.path.exists(results_dir):
    result_files = glob.glob(os.path.join(results_dir, "*.txt"))
    
    print("Average Accuracies:")
    print("="*60)
    for f in sorted(result_files):
        avg_acc = calculate_avg_accuracy(f)
        if avg_acc is not None:
            filename = os.path.basename(f)
            print(f"{filename:60s} {avg_acc:.4f}%")
    
    # Overall average
    all_accs = [calculate_avg_accuracy(f) for f in result_files]
    all_accs = [a for a in all_accs if a is not None]
    if all_accs:
        print("="*60)
        print(f"{'Overall Average:':60s} {np.mean(all_accs):.4f}%")
        print(f"{'Std Dev:':60s} {np.std(all_accs):.4f}%")
else:
    print(f"Results directory not found: {results_dir}")


Average Accuracies:
model_rdumb_baseline_20_transition+speed_1000_seed_43.txt    32.1429%
model_rdumb_baseline_20_transition+speed_1000_seed_44.txt    28.5156%
model_rdumb_baseline_20_transition+speed_1000_seed_45.txt    28.5362%
model_rdumb_baseline_20_transition+speed_2000_seed_43.txt    37.5615%
model_rdumb_baseline_20_transition+speed_2000_seed_44.txt    25.1302%
model_rdumb_baseline_20_transition+speed_2000_seed_45.txt    25.1302%
model_rdumb_baseline_20_transition+speed_5000_seed_43.txt    40.3125%
model_rdumb_baseline_20_transition+speed_5000_seed_44.txt    28.5156%
Overall Average:                                             30.7306%
Std Dev:                                                     5.2181%
